### EXTRACT DO BANDO DE DADOS RELACIONAL PARA UM ARQUIVO DATAFRAME CSV

In [13]:
import sqlite3
import pandas as pd

# Função para exportar dados do SQLite para CSV
def export_to_csv(database_name, csv_file):
    connection = sqlite3.connect(database_name)
    query = "SELECT * FROM reclamacoes"
    
    # Usando pandas para ler os dados do SQLite e exportar para CSV
    df = pd.read_sql_query(query, connection)
    df.to_csv(csv_file, index=False)
    
    connection.close()

# Nome do banco de dados SQLite
database_name = 'reclamacoes.db'

# Nome do arquivo CSV de saída
csv_file = 'reclamacoes_data.csv'
m
# Chamando a função para exportar os dados
export_to_csv(database_name, csv_file)

print(f'Dados exportados com sucesso para: {csv_file}')


Dados exportados com sucesso para: reclamacoes_data.csv


### LEITURA DO DATAFRAME

In [14]:
# Lendo o arquivo CSV
df = pd.read_csv('reclamacoes_data.csv')

df

,id,titulo,descricao,status,data
0,1,COBRANÇA [Editado pelo Reclame Aqui] NUM IMÓVE...,Tenho uma amiga que tem uma casa em Anchieta n...,Não respondida,Há 1 hora
1,2,Vivendo no escuro,Desde quarta feira 31/01/24 por volta das 16:2...,Não respondida,Há 1 hora
2,3,Falta de energia elétrica - Empresa Magia Anim...,"Como é fato publico e notório, a região da Ilh...",Não respondida,Há 2 horas
3,4,Contrato encerrado,segue o código de instalação*******Código do c...,Não respondida,Há 2 horas
4,5,Falta de luz e Recusa em fornecer os protocolos,Liguei pra empresa diversas vezes relatando a ...,Não respondida,Há 3 horas
...,...,...,...,...,...
515,516,"Idosa sem luz a 5 dias, diversas chamadas para...","Moro em Copacabana, com minha tia, idosa e com...",Respondida,Há 10 dias
516,517,Prédio sem energia há 2 dias,Sou morador do Prédio Edifício Residencial Mon...,NaN,Há 10 dias
517,518,Leitura feita por estimativa,Desde outubro/2023 a light vem fazendo a leitu...,Respondida,Há 10 dias
518,519,Poste energizado ( Risco a vida),"No dia 11 de dezembro de 2023, entrei em conta...",Respondida,Há 10 dias


### UTILIZANDO O MODELO NLTK (Natural Language Toolkit) PARA IDENTIFICAR POLARIDADE NA DESCRIÇÃO DA RECLAMAÇÃO

In [15]:
import nltk
nltk.download('vader_lexicon')

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


# Inicializando o analisador de sentimento VADER
analyzer = SentimentIntensityAnalyzer()

# Aplicando a análise de sentimento à coluna de descrição
df['sentimento'] = df['descricao'].apply(lambda x: analyzer.polarity_scores(x)['compound'])

# Classificando as polaridades de sentimento
df['sentimento'] = df['sentimento'].apply(lambda x: 'Positivo' if x >= 0.05 else ('Negativo' if x <= -0.05 else 'Neutro'))

# Exibindo o DataFrame com as novas colunas de sentimento
print(df[['descricao', 'sentimento']])


                                             descricao sentimento
0    Tenho uma amiga que tem uma casa em Anchieta n...   Negativo
1    Desde quarta feira 31/01/24 por volta das 16:2...     Neutro
2    Como é fato publico e notório, a região da Ilh...     Neutro
3    segue o código de instalação*******Código do c...   Positivo
4    Liguei pra empresa diversas vezes relatando a ...   Negativo
..                                                 ...        ...
515  Moro em Copacabana, com minha tia, idosa e com...   Positivo
516  Sou morador do Prédio Edifício Residencial Mon...   Negativo
517  Desde outubro/2023 a light vem fazendo a leitu...     Neutro
518  No dia 11 de dezembro de 2023, entrei em conta...   Negativo
519  Pedi o reset do ex email q tinha para poder ac...     Neutro

[520 rows x 2 columns]


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\felipe\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


#### UTILIZANDO TOKENS PARA IDENTIFICAR SE EXISTE A PALAVRA FALTA DE LUZ NO TÍTULO, CASO TENHA ESSA OCORRÊNCIA SERÁ IDENTIFICADO NA COLUNA "FALTA LUZ STATUS"

In [16]:
from nltk.tokenize import word_tokenize

# Função para verificar se a palavra "falta" ou "sem" está presente no título
def check_for_falta_ou_sem(title):
    # Tokenização
    tokens = word_tokenize(title.lower())
    
    # Verificando a presença da palavra "falta" ou "sem"
    if 'falta' in tokens:
        return 'falta de luz'
    elif 'sem' in tokens:
        return 'falta de luz'
    else:
        return 'normal'

# Aplicando a função à coluna de título
df['falta_luz_status'] = df['titulo'].apply(check_for_falta_ou_sem)

df

,id,titulo,descricao,status,data,sentimento,falta_luz_status
0,1,COBRANÇA [Editado pelo Reclame Aqui] NUM IMÓVE...,Tenho uma amiga que tem uma casa em Anchieta n...,Não respondida,Há 1 hora,Negativo,normal
1,2,Vivendo no escuro,Desde quarta feira 31/01/24 por volta das 16:2...,Não respondida,Há 1 hora,Neutro,normal
2,3,Falta de energia elétrica - Empresa Magia Anim...,"Como é fato publico e notório, a região da Ilh...",Não respondida,Há 2 horas,Neutro,falta de luz
3,4,Contrato encerrado,segue o código de instalação*******Código do c...,Não respondida,Há 2 horas,Positivo,normal
4,5,Falta de luz e Recusa em fornecer os protocolos,Liguei pra empresa diversas vezes relatando a ...,Não respondida,Há 3 horas,Negativo,falta de luz
...,...,...,...,...,...,...,...
515,516,"Idosa sem luz a 5 dias, diversas chamadas para...","Moro em Copacabana, com minha tia, idosa e com...",Respondida,Há 10 dias,Positivo,falta de luz
516,517,Prédio sem energia há 2 dias,Sou morador do Prédio Edifício Residencial Mon...,NaN,Há 10 dias,Negativo,falta de luz
517,518,Leitura feita por estimativa,Desde outubro/2023 a light vem fazendo a leitu...,Respondida,Há 10 dias,Neutro,normal
518,519,Poste energizado ( Risco a vida),"No dia 11 de dezembro de 2023, entrei em conta...",Respondida,Há 10 dias,Negativo,normal


#### SEGUNDO PROCESSO DE ETL, TRANFORMANDO AS DATAS EM UM FORMATO QUE A MAQUINA ENTENDA.

In [17]:
import pandas as pd
from datetime import datetime, timedelta
import re


# Função para converter "há [] hora" ou "há [] dias" para horas e dias
def converter_tempo(text):
    match = re.search(r'(\d+) (hora|dia)', text)
    if match:
        value, unit = match.groups()
        value = int(value)
        if unit == 'hora':
            return datetime.now() - timedelta(hours=value)
        elif unit == 'dia':
            return datetime.now() - timedelta(days=value)
    return datetime.now()

# Aplicando a função à coluna 'data'
df['data'] = df['data'].apply(converter_tempo).dt.floor('s')  # Removendo os milissegundos

# Exibindo o DataFrame atualizado
df.head(5)


,id,titulo,descricao,status,data,sentimento,falta_luz_status
0,1,COBRANÇA [Editado pelo Reclame Aqui] NUM IMÓVE...,Tenho uma amiga que tem uma casa em Anchieta n...,Não respondida,2024-02-04 13:55:56,Negativo,normal
1,2,Vivendo no escuro,Desde quarta feira 31/01/24 por volta das 16:2...,Não respondida,2024-02-04 13:55:56,Neutro,normal
2,3,Falta de energia elétrica - Empresa Magia Anim...,"Como é fato publico e notório, a região da Ilh...",Não respondida,2024-02-04 12:55:56,Neutro,falta de luz
3,4,Contrato encerrado,segue o código de instalação*******Código do c...,Não respondida,2024-02-04 12:55:56,Positivo,normal
4,5,Falta de luz e Recusa em fornecer os protocolos,Liguei pra empresa diversas vezes relatando a ...,Não respondida,2024-02-04 11:55:56,Negativo,falta de luz


In [24]:
# Converte a coluna 'data' para o formato de data do Pandas
df['data'] = pd.to_datetime(df['data'])

# Extrai somente a parte da data (ano, mês e dia)
df['data'] = df['data'].dt.date

df

,titulo,descricao,status,data,sentimento,falta_luz_status
id,,,,,,
1,COBRANÇA [Editado pelo Reclame Aqui] NUM IMÓVE...,Tenho uma amiga que tem uma casa em Anchieta n...,Não respondida,2024-02-04,Negativo,normal
2,Vivendo no escuro,Desde quarta feira 31/01/24 por volta das 16:2...,Não respondida,2024-02-04,Neutro,normal
3,Falta de energia elétrica - Empresa Magia Anim...,"Como é fato publico e notório, a região da Ilh...",Não respondida,2024-02-04,Neutro,falta de luz
4,Contrato encerrado,segue o código de instalação*******Código do c...,Não respondida,2024-02-04,Positivo,normal
5,Falta de luz e Recusa em fornecer os protocolos,Liguei pra empresa diversas vezes relatando a ...,Não respondida,2024-02-04,Negativo,falta de luz
...,...,...,...,...,...,...
516,"Idosa sem luz a 5 dias, diversas chamadas para...","Moro em Copacabana, com minha tia, idosa e com...",Respondida,2024-01-25,Positivo,falta de luz
517,Prédio sem energia há 2 dias,Sou morador do Prédio Edifício Residencial Mon...,NaN,2024-01-25,Negativo,falta de luz
518,Leitura feita por estimativa,Desde outubro/2023 a light vem fazendo a leitu...,Respondida,2024-01-25,Neutro,normal


In [19]:
df.to_csv('ETL-Reclamações.csv', index=False)
# Exiba o DataFrame
df

,titulo,descricao,status,data,sentimento,falta_luz_status
id,,,,,,
1,COBRANÇA [Editado pelo Reclame Aqui] NUM IMÓVE...,Tenho uma amiga que tem uma casa em Anchieta n...,Não respondida,2024-02-04 13:55:56,Negativo,normal
2,Vivendo no escuro,Desde quarta feira 31/01/24 por volta das 16:2...,Não respondida,2024-02-04 13:55:56,Neutro,normal
3,Falta de energia elétrica - Empresa Magia Anim...,"Como é fato publico e notório, a região da Ilh...",Não respondida,2024-02-04 12:55:56,Neutro,falta de luz
4,Contrato encerrado,segue o código de instalação*******Código do c...,Não respondida,2024-02-04 12:55:56,Positivo,normal
5,Falta de luz e Recusa em fornecer os protocolos,Liguei pra empresa diversas vezes relatando a ...,Não respondida,2024-02-04 11:55:56,Negativo,falta de luz
...,...,...,...,...,...,...
516,"Idosa sem luz a 5 dias, diversas chamadas para...","Moro em Copacabana, com minha tia, idosa e com...",Respondida,2024-01-25 14:55:56,Positivo,falta de luz
517,Prédio sem energia há 2 dias,Sou morador do Prédio Edifício Residencial Mon...,NaN,2024-01-25 14:55:56,Negativo,falta de luz
518,Leitura feita por estimativa,Desde outubro/2023 a light vem fazendo a leitu...,Respondida,2024-01-25 14:55:56,Neutro,normal
